In [1]:
import pandas as pd
import requests
import urllib.request 
from PIL import Image 
from io import BytesIO
from bs4 import BeautifulSoup
import os

In [2]:
# read file with links
gc_file = pd.read_excel('data/Wk33-34 Big Brand Sale Brief V1.xlsx', sheet_name = 'RD TEST')
gc_file.columns.values[0] = 'folder_name'
gc_file.head(5)

,folder_name,Image Location
0,FOOD9,https://www.petbarn.com.au/royal-canin-labrado...
1,FOOD9,https://www.petbarn.com.au/advance-delicate-tu...
2,FOOD9,https://www.petbarn.com.au/science-diet-feline...
3,FOOD9,https://www.petbarn.com.au/black-hawk-grain-fr...
4,LITTER5,https://www.petbarn.com.au/freshco-clay-clumpi...


In [7]:
# Function to download an image
def download_image(url, folder_name, image_counters):
    headers = {
        'Cache-Control': 'no-cache',
        'Pragma': 'no-cache'
    }
    response = requests.get(url, headers = headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    img_tag = soup.find('img', {'alt':'main product photo'})
    try:
        img_url = img_tag['src']
        img_response = requests.get(img_url)
        if img_response.status_code == 200:
            # Create the folder if it does not exist
            os.makedirs(f'Trade Sales/{folder_name}', exist_ok=True)
            
            # Initialize the image counter for the folder if not already set
            if folder_name not in image_counters:
                image_counters[folder_name] = 1  # Start counting from 1
            
            # Get the current image counter for the folder
            img_counter = image_counters[folder_name]
            img_path = os.path.join('Trade Sales', folder_name, f'image_{img_counter}.jpg')
            
            with open(img_path, 'wb') as f:
                f.write(img_response.content)
            print(f'Downloaded {img_path} from {url}')
            # Increment the image counter for the folder
            image_counters[folder_name] += 1
        else:
            print(f'Failed to download image from {url}')
    except TypeError:
        if url[0] == 'O' or url[0] == 'W':
            print(f'{folder_name}: {url} is a server file path, please access manually')
        else:
            print(f'Likely URL gave 404. Please check {folder_name}: {url}')
        pass

# Initialize dictionary to store image counters per folder
image_counters = {}

# Loop through the DataFrame and download each image
for index, row in gc_file.iterrows():
    folder_name = row['folder_name']
    url = row['Image Location']
    download_image(url.strip(), folder_name.strip(), image_counters)

Downloaded Trade Sales\FOOD9\image_1.jpg from https://www.petbarn.com.au/royal-canin-labrador-dog-food?sku=30327
Downloaded Trade Sales\FOOD9\image_2.jpg from https://www.petbarn.com.au/advance-delicate-tuna-adult-cat-food-85gx7-c76ab7
Downloaded Trade Sales\FOOD9\image_3.jpg from https://www.petbarn.com.au/science-diet-feline-oral-care-cat-food?sku=132843
Downloaded Trade Sales\FOOD9\image_4.jpg from https://www.petbarn.com.au/black-hawk-grain-free-chicken-adult-dog-can-100gx9
Downloaded Trade Sales\LITTER5\image_1.jpg from https://www.petbarn.com.au/freshco-clay-clumping-cat-litter?sku=141868
Downloaded Trade Sales\LITTER5\image_2.jpg from https://www.petbarn.com.au/cature-tofu-clumping-milky-cat-litter?sku=144701
Most likely URL gave 404. Please check LITTER5: https://www.petbarn.com.au/rufus-coco-wee-kitty-clumping-corn-cat-litter?sku=144994
Most likely URL gave 404. Please check LITTER5: https://www.petbarn.com.au/ezi-lockodour-natural-mineral-zeolite-cat-litter-pellets-2l
Downloa